In [1]:
import sys
import time
import pandas as pd
from app.builtin import *

sys.path.append('../app')  # for let gsdmm to work

Using TensorFlow backend.


In [2]:
dataset = '../data/20ng.txt'
labels = '../data/20ng_labels.txt'

In [3]:
with open(labels, "r") as datafile:
    true_topics = [line.rstrip() for line in datafile if line]
true_topics[0:10]

['rec.autos',
 'comp.sys.mac.hardware',
 'comp.sys.mac.hardware',
 'comp.graphics',
 'sci.space',
 'talk.politics.guns',
 'sci.med',
 'comp.sys.ibm.pc.hardware',
 'comp.os.ms-windows.misc',
 'comp.sys.mac.hardware']

In [4]:
models= {
    'ntm': NtmModel,
    'lda': LdaModel,
    'lftm': LftmModel,
    'gsdmm': GsdmmModel
}

In [5]:
results = {}
for name, model in models.items():
    start = time.time()
    m = model()
    print(name)
    results[name] = m.get_corpus_predictions()
    dur = time.time() - start
    print('Done in ' + str(dur))

W0408 16:48:56.695528 4631326144 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0408 16:48:56.841341 4631326144 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ntm


W0408 16:48:57.148264 4631326144 module_wrapper.py:139] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Done in 1.853410005569458
lda
Done in 0.5259079933166504
lftm
Done in 0.25112485885620117
gsdmm
Done in 29.16367793083191


In [6]:
df = pd.DataFrame()
df['true'] = true_topics
for name, model in models.items():
    r = results[name]
    for i in range(0,3):
        key, value = zip(*[x[i] if len(x) > i else (-1, -1) for x in r])
        df['%s_topic_%d'%(name,i)] = key
        df['%s_score_%d'%(name,i)] = value
df

,true,ntm_topic_0,ntm_score_0,ntm_topic_1,ntm_score_1,ntm_topic_2,ntm_score_2,lda_topic_0,lda_score_0,lda_topic_1,...,lftm_topic_1,lftm_score_1,lftm_topic_2,lftm_score_2,gsdmm_topic_0,gsdmm_score_0,gsdmm_topic_1,gsdmm_score_1,gsdmm_topic_2,gsdmm_score_2
0,rec.autos,11,0.227447,4,0.202119,5,0.163517,0,0.001044,1,...,3,0.220588,16,0.073529,3,0.230260,5,0.128973,0,0.112503
1,comp.sys.mac.hardware,14,0.249227,18,0.174287,0,0.165302,0,0.000661,1,...,8,0.146067,17,0.089888,3,0.200210,5,0.122734,1,0.111960
2,comp.sys.mac.hardware,7,0.333553,14,0.272124,5,0.116028,0,0.000367,1,...,10,0.160714,7,0.089286,3,0.200210,5,0.122734,1,0.111960
3,comp.graphics,16,0.254609,0,0.196284,2,0.137768,0,0.000881,1,...,19,0.098592,6,0.084507,3,0.237918,5,0.130291,0,0.115251
4,sci.space,5,0.292701,0,0.186552,10,0.141531,0,0.000629,1,...,1,0.134021,2,0.134021,3,0.292335,5,0.136717,0,0.133362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,sci.med,7,0.601212,8,0.236097,2,0.131402,0,0.000357,1,...,10,0.085714,3,0.080000,3,0.307924,0,0.138086,5,0.137662
11310,comp.sys.mac.hardware,15,0.339457,2,0.160176,19,0.155714,0,0.000862,1,...,17,0.126582,13,0.101266,3,0.150933,1,0.112778,5,0.108369
11311,comp.sys.ibm.pc.hardware,9,0.211234,14,0.181800,12,0.162045,0,0.000748,1,...,8,0.202532,3,0.088608,3,0.253358,5,0.132627,0,0.120643
11312,comp.graphics,4,0.256770,7,0.198837,16,0.172548,0,0.000558,1,...,19,0.092593,2,0.083333,3,0.245620,5,0.131508,0,0.117966


In [7]:
df.index.name = 'id'
df.to_csv('./out/predictions_20ng.csv')

In [9]:
import json 

for name, model in models.items():
    m = model()
    with open('./out/name+'_topics.json', 'w') as file:
        json.dump(m.topics(), file)